In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps
import json
import urllib

from config import api_key
import gkey

taxi_output_data_file = "output_data/rides_june_2015.csv"
rideshare_output_data_file = "output_data/hv_rideshare_june_2019.csv"

#Import Manhattan zones df
manhattan_zones = pd.read_csv("../../../Desktop/NYC Taxi & FHV Data/manhattan_zones.csv")
manhattan_zones.head()

ModuleNotFoundError: No module named 'gkey'

# Yellow Taxi June 2015 Data

In [2]:
yellow_taxi_2015_06 = "../../../Desktop/NYC Taxi & FHV Data/yellow_tripdata_2015-06.csv"
yt_june_2015_df = pd.read_csv(yellow_taxi_2015_06)

In [3]:
clean_yt_june_2015_df= yt_june_2015_df.drop(columns=['VendorID','extra','store_and_fwd_flag','payment_type','mta_tax','tolls_amount','improvement_surcharge'])
clean_yt_june_2015_df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount
0,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80
1,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30
2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00
3,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16
4,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30


In [4]:
#Remove rows where passenger count is zero
clean_yt_june_2015_df = clean_yt_june_2015_df.loc[clean_yt_june_2015_df["passenger_count"] != 0]

In [ ]:
#Split tpep_pickup_datetime column into date and time
cut_dates = clean_yt_june_2015_df["tpep_pickup_datetime"].str.split(" ", n = 2, expand = True)
clean_yt_june_2015_df["tpep_pickup_datetime"] = cut_dates[0]
clean_yt_june_2015_df["pickup_time"] = cut_dates[1]

clean_yt_june_2015_df.head() 

KeyError: 1

In [6]:
#Isolate month from date and create new column with month number
cut_month = clean_yt_june_2015_df['tpep_pickup_datetime'].str.split("-", expand = True)

clean_yt_june_2015_df["Month"] = cut_month[1].astype(int)
clean_yt_june_2015_df["Month"]
clean_yt_june_2015_df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount,pickup_time,Month
0,2015-06-02,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,-73.974754,40.754093,17.0,0.00,17.80,11:19:29,6
1,2015-06-02,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,-73.978539,40.761909,6.5,1.00,8.30,11:19:30,6
2,2015-06-02,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,-73.990273,40.745438,8.0,2.20,11.00,11:19:31,6
3,2015-06-02,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,-73.971527,40.760330,13.5,2.86,17.16,11:19:31,6
4,2015-06-02,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,-73.982162,40.758999,9.5,0.00,10.30,11:19:32,6


In [7]:
#Check to ensure that only data for June is in data frame
clean_yt_june_2015_df.groupby("Month").count()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount,total_amount,pickup_time
Month,,,,,,,,,,,,,
6,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381,12323381


In [8]:
#Limit pickup locations to longitude and latitudes in Manhattan
manhattan_yt_june_2015_df = clean_yt_june_2015_df.loc[(clean_yt_june_2015_df["pickup_latitude"]<= 40.737052) & (clean_yt_june_2015_df["pickup_latitude"] >= 40.722288) & 
                               (clean_yt_june_2015_df["pickup_longitude"]<= -73.972107) & (clean_yt_june_2015_df["pickup_longitude"] >= -74.003929)]

#Limit dropoff locations to longitude and latitudes in Manhattan
manhattan_yt_june_2015_df = clean_yt_june_2015_df.loc[(clean_yt_june_2015_df["dropoff_latitude"]<= 40.737052) & (clean_yt_june_2015_df["dropoff_latitude"] >= 40.722288) & 
                               (clean_yt_june_2015_df["dropoff_longitude"]<= -73.972107) & (clean_yt_june_2015_df["dropoff_longitude"] >= -74.003929)]

manhattan_yt_june_2015_df.to_csv("../../NYC Taxi & FHV Data/clean_csvs/taxi_june_2015.csv")

# High Voume FHV June 2019 Data

In [9]:
#Import original csv
hi_vol_for_hire_veh_2019_06 = "../../../Desktop/NYC Taxi & FHV Data/fhvhv_tripdata_2019-06.csv"
hvfhv_june_2019_df = pd.read_csv(hi_vol_for_hire_veh_2019_06)
hvfhv_june_2019_df.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0002,B03035,2019-06-01 00:40:55,2019-06-01 01:03:58,256,87,NaN
1,HV0005,B02510,2019-06-01 00:04:47,2019-06-01 00:25:53,163,265,NaN
2,HV0005,B02510,2019-06-01 00:48:00,2019-06-01 00:59:49,132,216,1.0
3,HV0002,B03035,2019-06-01 00:59:25,2019-06-01 01:04:07,152,116,NaN
4,HV0003,B02682,2019-06-01 00:05:11,2019-06-01 00:23:35,148,233,NaN


In [10]:
#Drop SR_Flag column
clean_hvfhv_june_2019_df= hvfhv_june_2019_df.drop(columns=['SR_Flag'])
clean_hvfhv_june_2019_df.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID
0,HV0002,B03035,2019-06-01 00:40:55,2019-06-01 01:03:58,256,87
1,HV0005,B02510,2019-06-01 00:04:47,2019-06-01 00:25:53,163,265
2,HV0005,B02510,2019-06-01 00:48:00,2019-06-01 00:59:49,132,216
3,HV0002,B03035,2019-06-01 00:59:25,2019-06-01 01:04:07,152,116
4,HV0003,B02682,2019-06-01 00:05:11,2019-06-01 00:23:35,148,233


In [11]:
#Drop rows with incomplete data
clean_hv_fhv_june_2019_df= clean_hvfhv_june_2019_df.dropna(how = "any")
clean_hv_fhv_june_2019_df

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID
0,HV0002,B03035,2019-06-01 00:40:55,2019-06-01 01:03:58,256,87
1,HV0005,B02510,2019-06-01 00:04:47,2019-06-01 00:25:53,163,265
2,HV0005,B02510,2019-06-01 00:48:00,2019-06-01 00:59:49,132,216
3,HV0002,B03035,2019-06-01 00:59:25,2019-06-01 01:04:07,152,116
4,HV0003,B02682,2019-06-01 00:05:11,2019-06-01 00:23:35,148,233
...,...,...,...,...,...,...
21001906,HV0005,B02510,2019-06-30 23:30:47,2019-07-01 00:07:37,60,230
21001907,HV0003,B02765,2019-06-30 23:25:12,2019-07-01 00:05:44,138,246
21001908,HV0005,B02510,2019-06-30 23:11:34,2019-06-30 23:23:08,225,61
21001909,HV0003,B02764,2019-06-30 23:06:29,2019-06-30 23:34:04,68,141


In [12]:
#Split pickup_datetime column into date and time
cut_dates = clean_hv_fhv_june_2019_df["pickup_datetime"].str.split(" ", n =2, expand = True)
clean_hv_fhv_june_2019_df["pickup_datetime"] = cut_dates[0]
clean_hv_fhv_june_2019_df["pickup_time"] = cut_dates[1]

clean_hv_fhv_june_2019_df

/var/folders/w4/cz88rjkj0lg25jggxn27s8l40000gn/T/ipykernel_53813/2563255581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_hv_fhv_june_2019_df["pickup_datetime"] = cut_dates[0]
/var/folders/w4/cz88rjkj0lg25jggxn27s8l40000gn/T/ipykernel_53813/2563255581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_hv_fhv_june_2019_df["pickup_time"] = cut_dates[1]


,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_time
0,HV0002,B03035,2019-06-01,2019-06-01 01:03:58,256,87,00:40:55
1,HV0005,B02510,2019-06-01,2019-06-01 00:25:53,163,265,00:04:47
2,HV0005,B02510,2019-06-01,2019-06-01 00:59:49,132,216,00:48:00
3,HV0002,B03035,2019-06-01,2019-06-01 01:04:07,152,116,00:59:25
4,HV0003,B02682,2019-06-01,2019-06-01 00:23:35,148,233,00:05:11
...,...,...,...,...,...,...,...
21001906,HV0005,B02510,2019-06-30,2019-07-01 00:07:37,60,230,23:30:47
21001907,HV0003,B02765,2019-06-30,2019-07-01 00:05:44,138,246,23:25:12
21001908,HV0005,B02510,2019-06-30,2019-06-30 23:23:08,225,61,23:11:34
21001909,HV0003,B02764,2019-06-30,2019-06-30 23:34:04,68,141,23:06:29


In [13]:
#Designate list of Manhattan taxi zones
manhattan_area = manhattan_zones["LocationID"]

#Limit pickup/dropoff areas to Manhattan
for zone in manhattan_area:
    manhattan_hv_fhv_june_2019 = clean_hv_fhv_june_2019_df.loc[(clean_hv_fhv_june_2019_df["PULocationID"] == zone) | 
                                                                (clean_hv_fhv_june_2019_df["DOLocationID"] == zone)
                                                                  ]


manhattan_hv_fhv_june_2019

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_time
90,HV0004,B02800,2019-06-01,2019-06-01 00:16:41,237,263,00:09:57
143,HV0003,B02869,2019-06-01,2019-06-01 00:37:38,45,263,00:20:37
171,HV0003,B02884,2019-06-01,2019-06-01 00:41:14,170,263,00:30:42
199,HV0003,B02764,2019-06-01,2019-06-01 00:39:28,263,236,00:34:54
273,HV0003,B02835,2019-06-01,2019-06-01 01:18:08,179,263,00:34:46
...,...,...,...,...,...,...,...
21001465,HV0005,B02510,2019-06-30,2019-06-30 23:30:13,255,263,23:03:57
21001521,HV0004,B02800,2019-06-30,2019-06-30 23:52:03,263,179,23:37:57
21001585,HV0003,B02764,2019-06-30,2019-06-30 23:34:53,263,265,23:07:00
21001726,HV0005,B02510,2019-06-30,2019-06-30 23:32:39,263,162,23:24:17


In [14]:
#Export to clean csvs
manhattan_hv_fhv_june_2019.to_csv("../../NYC Taxi & FHV Data/clean_csvs/HighVol_june_2019.csv")